In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import tarfile
from six.moves import urllib
import matplotlib.pyplot as plt
%matplotlib inline

import os
from PIL import Image
import pathlib
import csv

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import regularizers

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

#Confusion Matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset_path = '/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_30_sec.csv'

In [ ]:
seed = 7
np.random.seed(seed)

master_data = pd.read_csv(dataset_path) 
# master_data = master_data.drop(master_data.columns[range(21,59)], axis = 1)
master_data = master_data.drop("filename", axis = 1)

labels = ['blues',
          'classical',
          'country',
          'disco',
          'hiphop',
          'jazz',
          'metal',
          'pop',
          'reggae',
          'rock']

master_data.head()

In [ ]:
master_data.info()

In [ ]:
master_dataset = master_data.values
X = master_dataset[:,0:58].astype(float)
Y = master_dataset[:,58]
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy = np_utils.to_categorical(encoded_Y)
dummy[0]

In [ ]:
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
def baseline_model(optimizer='adam', init='glorot_uniform'):
  # create model
  model = Sequential()
  model.add(Dense(58, kernel_initializer=init, activation='relu'))
  model.add(Dense(29, kernel_initializer=init, activation='relu'))
  # model.add(Dense(20, kernel_initializer=init, activation='relu'))
  model.add(Dense(10, kernel_initializer=init, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

I used GridSearch Cross Validation to figure out the best fit for batch size, epochs, optimization and initalisation. I've used the results of that as my final values for the cross_val_score function.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, dummy, test_size=0.33, random_state=7)
model = KerasClassifier(build_fn=baseline_model, verbose=0)
# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [15, 30, 45]
batches = [5, 10, 20]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
model = KerasClassifier(build_fn=baseline_model, epochs=30, batch_size=5, verbose=0)
kfold = KFold(n_splits=8, shuffle=True,  random_state=seed)
results = cross_val_score(model, X, dummy, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))